### 函数进阶使用方法

In [1]:
# lambda,filter,map
items = list(map(lambda x:x**2,filter(lambda x:x%2==0,range(1,10))))
items

[4, 16, 36, 64]

#### 装饰器函数

In [4]:
from functools import wraps
from time import time,sleep

In [13]:
def record(output):
    '''可以参数化的装饰器'''
    def decorator(func):
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            start = time()
            result = func(*args, **kwargs)
            output(func.__name__, time()-start)
            return result
        return wrapper
    
    return decorator

In [21]:
@record(print)
def pp():
    start = time()
    sleep(1)
    end = time()
    print('hello world!')
    print(end-start)
pp()

hello world!
1.0001490116119385
<built-in method __sizeof__ of function object at 0x000001F3777DDAB0> 1.0001826286315918


#### 混入(Mixin)

In [12]:
# 例子：自定义字典限制只有在指定的key不存在时才能在字典中设置键值对
class SetOnceMappingMixin:
    """自定义混入类"""
    __slots__ = ()
    def __setitem__(self, key, value):
        if key in self:
            raise KeyError(str(key)+' is already exist!')
        return super().__setitem__(key,value)

In [13]:
class SetOnceDict(SetOnceMappingMixin,dict):
    pass

In [14]:
my_dict = SetOnceDict()

In [15]:
try:
    my_dict['name'] = 'Jack'
    my_dict['name'] = 'Tom'
except KeyError:
    raise KeyError(str(my_dict.keys()) + 'has existed')

KeyError: "dict_keys(['name'])has existed"

In [16]:
print(my_dict)

{'name': 'Jack'}


#### 元编程和元类
#### 对象是通过类创建的，类是通过元类创建的，元类提供了创建类的元信息。所有的类都直接或间接的继承自`object`，所有的元类都直接或间接的继承自`type`

In [17]:
import threading

In [23]:
# 例子：用元类实现单例模式。
class SingletonMeta(type):
    
    def __init__(cls,*args,**kwargs):
       cls._instance = None
       cls._lock = threading.Lock()
       super().__init__(*args,**kwargs)
       
    def __call__(cls, *args, **kwargs):
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = super().__call__(*args,**kwargs)
        return cls._instance
class President(metaclass=SingletonMeta):
    pass

In [28]:
president = President()

In [30]:
president._instance = 10

In [31]:
president._instance

10

In [32]:
p2 = President()

In [35]:
print(president._instance)
print(p2._instance)
p2._instance = 20
print(president._instance)
print(p2._instance)

10
10
20
20


#### 迭代器和生成器
#### 迭代器是实现了迭代器协议的对象
#### Python中没有像`protocol`或`interface`这样的定义协议的关键字。
#### Python中用魔术方法表示协议。
#### `__iter__`和`__next__`魔术方法就是迭代器协议。

In [31]:
class Fib:
    """
    迭代器
    斐波那契数列
    """
    def __init__(self,num):
        self.num = num
        self.a,self.b = 0,1
        self.idx = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.idx < self.num:
            self.a,self.b = self.b,self.a+self.b
            self.idx += 1
            return self.a
        raise StopIteration()

In [7]:
fib = Fib(2)

In [8]:
iter_fib = iter(fib)

In [12]:
now_fib = next(iter_fib)
print(now_fib)

StopIteration: 

In [35]:
fib = Fib(5)

In [37]:
iter_fib = iter(fib)

In [29]:
for i in iter_fib:
    print(i,end=' ')

1 1 2 3 5 

In [102]:
# 生成器是语法简化版的迭代器
def fib_gen(num):
    """生成器"""
    a,b = 0,1
    for _ in range(num):
        a, b = b, a+b
        yield a

In [103]:
gen_fib = fib_gen(5)

In [104]:
type(gen_fib)

generator

In [105]:
for i in gen_fib:
    print(i,end=' ')

1 1 2 3 5 

#### 生成器进化为协程
#### 生成器对象可以使用`send()`方法发送数据，发送的数据会成为生成器函数中通过`yield`表达式获得的值。这样，生成器就可以作为协程使用，协程简单的说就是可以相互协作的子程序

In [132]:
def cal_avg():
    """流式计算平均值"""
    total,counter = 0,0
    avg_value = None
    while True:
        value = yield avg_value
        # print(value)
        # print(counter)
        total,counter = total + value, counter + 1
        avg_value = total/counter

In [133]:
cal_gen = cal_avg()

In [134]:
type(cal_gen)

generator

In [135]:
next(cal_gen)

In [136]:
for i in [10,20,30]:
    print(cal_gen.send(i))

10.0
15.0
20.0
